In [1]:
# Solution Ex. 50_new
#
# This solution is correct but less efficient than that based on UDF and one single "path" (ex50_newSQL.ipynb).


In [2]:
inputPath  = "/data/students/bigdata-01QYD/ex_data/Ex50_new/data/sensors.txt"
outputPath = "res_out_Ex50_newSQL_v2/"

In [3]:
# Create a DataFrame from persons.csv
readingDF = spark.read.load(inputPath,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "sensorId")\
.withColumnRenamed("_c1", "date")\
.withColumnRenamed("_c2", "PM10")

In [4]:
#debug 
readingDF.printSchema()
readingDF.show()

root
 |-- sensorId: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- PM10: double (nullable = true)

+--------+-------------------+----+
|sensorId|               date|PM10|
+--------+-------------------+----+
|      s1|2016-01-01 00:00:00|60.5|
|      s2|2016-01-01 00:00:00|30.1|
|      s3|2016-01-01 00:00:00|30.0|
|      s1|2016-01-02 00:00:00|60.2|
|      s2|2016-01-02 00:00:00|20.4|
|      s3|2016-01-02 00:00:00|60.0|
|      s1|2016-01-03 00:00:00|55.5|
|      s2|2016-01-03 00:00:00|52.5|
|      s3|2016-01-03 00:00:00|52.5|
+--------+-------------------+----+



In [5]:
# "register" readingDF
readingDF.createOrReplaceTempView("readings")

In [6]:
# Compute for each sensor the number of critical dates
criticalDatesPerSensorDF = \
spark.sql("""SELECT sensorId, COUNT(*) as numCriticalDates
FROM readings
WHERE PM10>50
GROUP BY sensorId""")

In [7]:
# debug
criticalDatesPerSensorDF.show()

+--------+----------------+
|sensorId|numCriticalDates|
+--------+----------------+
|      s2|               1|
|      s3|               2|
|      s1|               3|
+--------+----------------+



In [8]:
# Compute for each sensor the number of normal dates
normalDatesPerSensorDF = \
spark.sql("""SELECT sensorId, COUNT(*) as numNormalDates
FROM readings
WHERE PM10<=50
GROUP BY sensorId""")

In [9]:
# debug
normalDatesPerSensorDF.show()

+--------+--------------+
|sensorId|numNormalDates|
+--------+--------------+
|      s2|             2|
|      s3|             1|
+--------+--------------+



In [10]:
# "register" normalDatesPerSensorDF and normalDatesPerSensorDF
criticalDatesPerSensorDF.createOrReplaceTempView("criticalDF")
normalDatesPerSensorDF.createOrReplaceTempView("normalDF")

In [11]:
# Join the criticalDF with normalDF and keep the sensors with numCriticalDates>numNormalDates
# Use left outer join to keep the sensors that are always associated with critical dates. Those 
# sensors are present in criticalDF but not in normalDF.
# 
# Pay attention that we are interested in the sensors with numCriticalDates>numNormalDates or 
# with numNormalDates equal to NULL
selectedSensorsDF = \
spark.sql("""SELECT criticalDF.sensorId
FROM criticalDF LEFT OUTER JOIN normalDF ON criticalDF.sensorId=normalDF.sensorId
WHERE criticalDF.numCriticalDates>normalDF.numNormalDates 
   OR normalDF.numNormalDates IS NULL""")

In [12]:
# debug
selectedSensorsDF.show()

+--------+
|sensorId|
+--------+
|      s3|
|      s1|
+--------+



In [13]:
# Store the result in the output folder
selectedSensorsDF\
.write.csv(outputPath, header=False)